### 추천시스템

- tf-idf와 코사인유사도로 추천 시스템 구현
- 원리 : 유사한 내용 추천으로 유사도가 높은 것 찾기
   * 영화 줄거리가 비슷한 영화를 찾아주기

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearnex import patch_sklearn
patch_sklearn()

data = pd.read_csv('movies_metadata.csv', encoding='utf-8')

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
/tmp/ipykernel_14525/197946517.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('movies_metadata.csv', encoding='utf-8')


In [3]:
df_overreview = data[['original_title', 'overview']]
df_overreview.head()

,original_title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...


In [4]:
df_overreview.dropna(inplace=True, subset=['overview'])

/tmp/ipykernel_14525/2604716919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_overreview.dropna(inplace=True, subset=['overview'])


In [5]:
overeview_list = df_overreview['overview'].to_list()

# TF-IDF : 단어들의 값 계산
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(overeview_list)


In [6]:


# 코사인 유사도 : 두개 matrix 에 대한 비교를 진행
cosine_sim = cosine_similarity(tfidf_matrix[:20], tfidf_matrix[:20])
cosine_sim.shape, tfidf_matrix.shape

((20, 20), (44512, 75827))

In [7]:
# 영화 제목 입력 ==> 해당 영화의 인덱스
(df_overreview['original_title'] == 'Father of the Bride Part II').argmax()

4

In [8]:
### ==> 영화 제목 색인 생성
### 영화제목 : 인덱스
title_to_idx = dict(zip(df_overreview['original_title'], df_overreview.index))

### 원하는 영화 인덱스 찾기
select_idx = title_to_idx['The Dark Knight Rises']
select_idx

18252

In [9]:
# 자동으로 시놉시스의 내용을 분석하여 영화를 추천해주는 시스템
whattosearch = 'joker life story'
transformedsearch = tfidf.transform([whattosearch])


searchresult = []
for idx, x in enumerate(tfidf_matrix):
    searchresult.append([df_overreview['original_title'].iloc[idx], cosine_similarity(x, transformedsearch).flatten()[0]])

In [10]:
searchsorted = sorted(searchresult, key=lambda x: x[1], reverse=True)
searchsorted[:5]

[['Batman vs Dracula', 0.3730114223621601],
 ['Laughter in Paradise', 0.3162644136004201],
 ['The Defiant Ones', 0.3051432053219834],
 ['Batman: Assault on Arkham', 0.30289364203359226],
 ['Lego Batman: The Movie - DC Super Heroes Unite', 0.29401618776703264]]

In [11]:
df_overreview[df_overreview['original_title'] == searchsorted[0][0]]['overview'].tolist()

['Gotham City is terrorized not only by recent escapees Joker and Penguin, but by the original creature of the night, Dracula! Can Batman stop the ruthless vampire before he turns everyone in the city, including The Caped Crusader, Joker and Penguin, into his mindless minions?']

In [12]:
cosine_similarity(tfidf_matrix[0], )

array([[1.]])